In [139]:
import random
import requests

from collections import Iterable
from collections import Iterator
from itertools import islice
from itertools import chain

## 迭代器与可迭代对象

实现了迭代器协议的就是可迭代对象。

In [2]:
li = [1, 2, 3]
print(li.__iter__.__doc__)  # iterable
it = li.__iter__()
print(next(it))
it2 = iter(it)
print(next(it2))

Implement iter(self).
1
2


In [3]:
s = ""
print(s.__getitem__.__doc__)  # iterable
s = "abcd"
print(s.__getitem__(2))

Return self[key].
c


## 用户定义的迭代器和可迭代对象

In [4]:
class WeatherIterator(Iterator):
    """Can be used in next method"""
    def __init__(self, cities):
        self.cities = cities
        self.index = 0
        
    def __next__(self):
        if self.index == len(self.cities):
            raise StopIteration
        city = cities[self.index]
        self.index += 1
        return self.__weather(city)
        
    def __weather(self, city):
        r = requests.get("http://wthrcdn.etouch.cn/weather_mini?city={}".format(city))
        try:
            data = r.json()['data']['forecast'][0]
            return "{}: {}, {}".format(city, data['low'], data['high'])
        except TypeError or IndexError:
            return ""


In [6]:
cities = ["阿勒泰", "西安", "成都", "杭州"]
wi = WeatherIterator(cities)
print(list(wi))
print()
for each in iter(wi):
    print(each)

['阿勒泰: 低温 17℃, 高温 32℃', '西安: 低温 21℃, 高温 26℃', '成都: 低温 21℃, 高温 26℃', '杭州: 低温 26℃, 高温 35℃']



In [7]:
class WeatherIterable(Iterable):
    """Return a Iterator."""
    def __init__(self, cities):
        self.cities = cities
        
    def __iter__(self):
        return WeatherIterator(self.cities)

In [8]:
for x in WeatherIterable(cities):
    print(x)

阿勒泰: 低温 17℃, 高温 32℃
西安: 低温 21℃, 高温 26℃
成都: 低温 21℃, 高温 26℃
杭州: 低温 26℃, 高温 35℃


## 使用生成器函数生成可迭代对象

In [72]:
class PrimeNumbers:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        self.cur = start
        
    @staticmethod
    def is_prime(num):
        if num < 2:
            return False
        
        for i in range(2, num):
            if num % i == 0:
                return False
        
        return True
    
    def __iter__(self):
        return self
    
    def __next__(self):
        while not self.is_prime(self.cur):
            self.cur += 1
            
        if self.cur >= self.end + 1:
            raise StopIteration
        
        ret = self.cur
        self.cur += 1
        
        return ret

In [73]:
PrimeNumbers(1, 20).is_prime(11)

True

In [79]:
p = PrimeNumbers(1, 20)

In [80]:
list(p)

[2, 3, 5, 7, 11, 13, 17, 19]

In [93]:
def prime_numbers(n):
    """Prime numbers generator."""
    for i in range(2, n + 1):
        for j in range(2, i):
            if i % j == 0:
                break
        else:
            yield i

In [94]:
list(prime_numbers(20))

[2, 3, 5, 7, 11, 13, 17, 19]

## 反向迭代

In [101]:
'''
class FloatRange:
    def __init__(self, start, end, step):
        self.start = start
        self.end = end
        self.step = step
        
    def __iter__(self):
        return self
    
    def __reversed__(self):
        self.start, self.end = self.end, self.start
        self.step = -self.step
        return self
    
    def __next__(self):
        if self.start + self.step > self.end:
            raise StopIteration
        
        ret = self.start + self.step
        self.start += self.step
        return ret
'''

In [110]:
# list(FloatRange(1, 2, 0.1))

In [111]:
class FloatRange:
    def __init__(self, start, end, step=0.1):
        self.start = start
        self.end = end
        self.step = step 
        
    def __iter__(self):
        t=self.start
        while t <= self.end:
            yield t
            t += self.step 
            
    def __reversed__(self):
        t = self.end
        while t >= self.start:
            yield t
            t -= self.step


In [112]:
list(FloatRange(1, 2, 0.1))

[1,
 1.1,
 1.2000000000000002,
 1.3000000000000003,
 1.4000000000000004,
 1.5000000000000004,
 1.6000000000000005,
 1.7000000000000006,
 1.8000000000000007,
 1.9000000000000008]

In [113]:
list(reversed(FloatRange(1, 2, 0.1)))

[2,
 1.9,
 1.7999999999999998,
 1.6999999999999997,
 1.5999999999999996,
 1.4999999999999996,
 1.3999999999999995,
 1.2999999999999994,
 1.1999999999999993,
 1.0999999999999992]

## 迭代器切片

In [126]:
print(islice.__doc__)

islice(iterable, stop) --> islice object
islice(iterable, start, stop[, step]) --> islice object

Return an iterator whose next() method returns selected values from an
iterable.  If start is specified, will skip all preceding elements;
otherwise, start defaults to zero.  Step defaults to one.  If
specified as another value, step determines how many values are 
skipped between successive calls.  Works like a slice() on a list
but returns an iterator.


In [129]:
with open("../LICENSE", "r") as f:
    it = islice(f, 10, 15)
    for line in it:
        print(line)



The above copyright notice and this permission notice shall be included in all

copies or substantial portions of the Software.



THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR



In [132]:
l = range(10)
it = iter(l)
part = islice(it, 2, 4)
for each in part:
    print(each)

2
3


In [133]:
for each in it:
    print(each)

4
5
6
7
8
9


## 在一个 For 循环中迭代多个对象

In [135]:
math = [random.randint(0, 100) for _ in range(10)]
chinese = [random.randint(0, 100) for _ in range(10)]
english = [random.randint(0, 100) for _ in range(10)]

In [136]:
for m, c, e in zip(math, chinese, english):
    print(c + m + e)

45
108
99
144
108
83
21
184
191
156


In [137]:
english1 = [random.randint(0, 100) for _ in range(10)]
english2 = [random.randint(0, 100) for _ in range(10)]
english3 = [random.randint(0, 100) for _ in range(10)]
english4 = [random.randint(0, 100) for _ in range(10)]

In [140]:
print(chain.__doc__)

chain(*iterables) --> chain object

Return a chain object whose .__next__() method returns elements from the
first iterable until it is exhausted, then elements from the next
iterable, until all of the iterables are exhausted.


In [143]:
ret = []
for g in chain(english1, english2, english3, english4):
    if g > 90:
        ret.append(g)

In [144]:
ret

[99, 96, 96, 94, 97]